# 目的
- ブログ公開用にデータを事前に前処理する
    - 名前の隠蔽

In [1]:
import os, sys
import numpy as np
import pandas as pd

import ChocoUtils as cu

In [2]:
data = cu.get_data(db_file='../../data/choco-ball.db', filter_str="campaign is not 1 and taste is not 10")
data = data.drop(columns=['shop', 'factory'])

SELECT measure_date, best_before, prd_number, weight, box_weight, ball_number, factory, shop, angel, campaign, taste, buyer, (weight - box_weight), (weight - box_weight)/ball_number FROM measurement WHERE campaign is not 1 and taste is not 10;
Shape of MeasurementData(record_num, n_columns) : (290, 14)


In [4]:
# 銀のエンゼルのフラグデータ
data['silver'] = [1 if a == 1 else 0 for a in data['angel'].values]
# 金のエンゼル
data['gold'] = [1 if a == 2 else 0 for a in data['angel'].values]

# 購入者の調整
data['buyer'] = ['ピシャイ' if a in 'ピシャイ/佐藤' else a for a in data['buyer'].values]


In [5]:
# 購入者を限定する
data = data.query('buyer in ["ピシャイ", "西田", "時田"]')

grp = data.groupby(['buyer'])
print(grp.count()[['net_weight']])

       net_weight
buyer            
ピシャイ           80
時田            142
西田             59


In [7]:
name_dict = {'ピシャイ':'B', '時田':'A', '西田':'C'}

In [11]:
data['buyer'] = [name_dict[nm] for nm in data['buyer']]

In [12]:
data.head()

,measure_date,best_before,prd_number,weight,box_weight,ball_number,angel,campaign,taste,buyer,net_weight,mean_weight,silver,gold
2,2017-12-10,2018-09-01,CA9,31.103,4.776,15,0,2,1,A,26.327,1.755133,0,0
3,2017-12-10,2018-09-01,CA9,31.477,4.767,15,0,2,1,A,26.710,1.780667,0,0
4,2017-12-10,2018-09-01,CA9,31.615,4.777,15,0,2,1,A,26.838,1.789200,0,0
5,2017-12-10,2018-09-01,CA9,31.212,4.785,15,0,2,1,A,26.427,1.761800,0,0
6,2017-12-10,2018-09-01,CA9,31.394,4.819,15,0,2,1,A,26.575,1.771667,0,0


In [14]:
data.groupby(['buyer']).count()

,measure_date,best_before,prd_number,weight,box_weight,ball_number,angel,campaign,taste,net_weight,mean_weight,silver,gold,buyer_idx
buyer,,,,,,,,,,,,,,
A,142,142,142,142,142,142,142,142,142,142,142,142,142,142
B,80,80,80,80,80,80,80,80,80,80,80,80,80,80
C,59,59,59,59,59,59,59,59,59,59,59,59,59,59


In [16]:
data.to_csv('angel_data_for_publish.csv', index=False)

## 基礎集計

In [17]:
data.groupby(['buyer']).count()['taste']

buyer
A    142
B     80
C     59
Name: taste, dtype: int64

In [18]:
data.groupby(['buyer', 'silver']).count()['taste']

buyer  silver
A      0         136
       1           6
B      0          75
       1           5
C      0          57
       1           2
Name: taste, dtype: int64

In [21]:
print('total:', data.shape[0])
print('total(silver):', data['silver'].sum())
print('MLE:', data['silver'].sum()/data.shape[0])

total: 281
total(silver): 13
MLE: 0.046263345195729534
